In [1]:
from bats_label_model import BatsModel
import numpy as np
import pickle

from sklearn.utils import shuffle


2023-11-28 12:44:53.588984: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 12:44:53.624281: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 12:44:53.624826: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 12:44:54.350011: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


resources get initialised


Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
model = BatsModel("NNSeval-1249")

100%|██████████| 100/100 [00:07<00:00, 13.28epoch/s]
/workspace/pruning_lfs.py:67: RuntimeWarning: invalid value encountered in scalar divide
  merged_data.append([index, polarity, cov_simp, cov_src, precision, 1 - precision, cov_simp+cov_src, abs(cov_simp - cov_src), abs(cov_simp - cov_src)/(cov_simp+cov_src)])
/usr/local/lib/python3.8/site-packages/snorkel/labeling/model/label_model.py:387: RuntimeWarning: divide by zero encountered in divide
  return np.clip(accs / self.coverage, 1e-6, 1.0)


In [3]:
X, y = np.concatenate([model.simp_labels, model.src_labels]), np.array([0]*len(model.simp_labels) + [1]*len(model.src_labels))
X, y = shuffle(X, y, random_state=42)
sel_indices = np.array(model.lfs_vec).nonzero()[0]
opt_X = np.take(X, sel_indices, axis=1)

print(opt_X.shape)

clf.score(opt_X, y)

AttributeError: 'BatsModel' object has no attribute 'lfs_vec'

In [ ]:
model.lfs_vec.shape[0]

In [ ]:
with open("/workspace/datasets/eval_simp_labels.pkl", 'rb') as f:
    eval_simp = pickle.load(f)

In [ ]:
with open("workspace/datasets/final_combined_with_index.pkl", 'rb') as f:
    dataset = pickle.load(f)

In [ ]:
dataset['simplified_snt'] = dataset['simp']
dataset['source_snt'] = dataset['src']

In [ ]:
selected_dataset = "MTurkSF"
dataset = dataset[dataset['ds_id'] == selected_dataset]

In [ ]:
dataset

In [ ]:
for i in range(30):
    t1 = dataset.iloc[1].src
    print(model.calc_score(t1))

In [ ]:
def foo1(x=[]):
    x.append("a")
    print(x)

In [ ]:
foo1()

In [ ]:
t1_bin =model.transform_to_bin_vec(t1)

In [ ]:
t1_bin

In [ ]:
weights

In [ ]:
s_val = 0
ns_val = 0

for i in range(len(t1_bin[0])):
    if t1_bin[0][i] == 0:
        s_val += weights[i]
    elif t1_bin[0][i] == 1:
        ns_val += weights[i]

In [ ]:
s_val_p = s_val / (s_val + ns_val)

In [ ]:
ns_val_p = ns_val / (s_val + ns_val)

In [ ]:
ns_val_p

In [ ]:
from snorkel.labeling import labeling_function, PandasLFApplier, LabelingFunction
from snorkel.labeling.apply.dask import PandasParallelLFApplier, DaskLFApplier
import pandas as pd
import dask.dataframe as dd
import numpy as np
import pickle
import time

def is_odd(x):
    time.sleep(0.01)
    return 1 if x.val % 2 == 1 else 0

test_lfs = [LabelingFunction(f"lf_{i}", is_odd) for i in range(6)]

In [ ]:
rand_ints = np.random.randint(100, size=500)
df = pd.DataFrame(dict(val=rand_ints))
ddf = dd.from_pandas(df, npartitions=6)

In [ ]:
applier = PandasLFApplier(lfs)
labels = applier.apply(pd.DataFrame(dict(val=rand_ints)))

In [ ]:
applier_p = PandasParallelLFApplier(lfs)
labels = applier_p.apply(pd.DataFrame(dict(val=rand_ints)), n_parallel=6)

In [ ]:
dask_applier = DaskLFApplier(lfs)

In [ ]:
labels = dask_applier.apply(df = ddf)

In [ ]:
#try it own multiprocessing
from multiprocessing import Process


def apply_lfs(df, lfs, i, size):
    applier = PandasLFApplier(lfs)
    labels = applier.apply(df[i:i+size])
    pickle.dump(open(f"./test_{i}.pkl", "wb"), labels)


In [ ]:
procs = []

start = time.perf_counter()

for i in range(5):
    print(f"start process {i}")
    proc = Process(target=apply_lfs, args=(df[i:i+100], test_lfs))
    procs.append(proc)
    proc.start()


for proc in procs:
    proc.join()

end = time.perf_counter()
print(f"Duration: {np.round(end-start, 2)} seconds")